# Exploratory and Feature Engineering

In this notebook, we are performing exploratory data analysis and feature engineering to the data sets. We have many features which are discrete and we have to perform feature engineer on these features to change them to numerical so that they can be fitted to our model for evaluation

Contents:
- [Import Library and Load Data](#Import-Library-and-Load-Data)
- [Feature Engineering](#Feature-Engineering)
- [Final Check and Saving Data Frame](Final-Check-and-Saving-Data-Frame)

## Import Library and Load Data

In [1]:
#importing libraries

import pandas as pd
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
train = pd.read_csv('../datasets/train_cleansed.csv')
test = pd.read_csv('../datasets/test_cleansed.csv')

## Feature Engineering

### Sale Price

We shall take a look at the sale price. Primarily, we should see that there are no negative value or illogical values in the columns

In [3]:
train['SalePrice'].describe()

count      2048.000000
mean     181484.252441
std       79248.657891
min       12789.000000
25%      129837.500000
50%      162500.000000
75%      214000.000000
max      611657.000000
Name: SalePrice, dtype: float64

There are no obvious errors from this stage of the analysis

### ID and PID

We will create a function to return the number of unique values in the columns and also the number of rows in the dataframe to confirm that all values in the column are unique

In [4]:
# function to return the unique values and row of the column 

def check_unique(df, column_list):
    for i in column_list:
        rows = df.shape[0] # this gives the number of rows in the dataframe
        unique = len(df[i].unique()) # this gives the length of unique values in the column
        print("Number of unique {} is {} \nNumber of rows is {}.".format(i, unique, rows))

In [5]:
column_list = ['Id', 'PID']
check_unique(train, column_list)

Number of unique Id is 2048 
Number of rows is 2048.
Number of unique PID is 2048 
Number of rows is 2048.


In [6]:
check_unique(test, column_list)

Number of unique Id is 879 
Number of rows is 879.
Number of unique PID is 879 
Number of rows is 879.


From the above, we can see that all Id and PID are uniques. The number of unique values for each column is the same as the number of rows in the dataframe

### Rest of  Columns

We have 78 more columns to explore. We shall break them down into data types, then look at them. This function was created in notebook 1, we shall use them again in this notebook

In [7]:
# function to get data type for each column and display columns with same data types

def get_columns_dtype(dataframe):
    object_col = []
    integer_col = []
    float_col = []
    
    object_type = dataframe.select_dtypes([np.object]).columns
    integer_type = dataframe.select_dtypes([np.int64, np.uint8]).columns 
    float_type = dataframe.select_dtypes([np.float64]).columns
    
    for object_dtype in object_type:
        object_col.append(object_dtype)
    
    for integer_dtype in integer_type:
        integer_col.append(integer_dtype)
    
    for float_dtype in float_type:
        float_col.append(float_dtype)
        
        
    print('Columns with Object Data Type are: \n{}'.format(object_col))
    print('Total number of object columns are : {}'.format(len(object_col)))
    print('')
    print('Columns with Integer Data Type are: \n{}'.format(integer_col))
    print('Total number of integer columns are : {}'.format(len(integer_col)))
    print('')
    print('Columns with Float Data Type are: \n{}'.format(float_col))
    print('Total number of float columns are : {}'.format(len(float_col)))

In [8]:
get_columns_dtype(train)

Columns with Object Data Type are: 
['MS Zoning', 'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type', 'House Style', 'Roof Style', 'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2', 'Heating', 'Heating QC', 'Central Air', 'Electrical', 'Kitchen Qual', 'Functional', 'Fireplace Qu', 'Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond', 'Paved Drive', 'Pool QC', 'Fence', 'Misc Feature', 'Sale Type']
Total number of object columns are : 42

Columns with Integer Data Type are: 
['Id', 'PID', 'MS SubClass', 'Lot Area', 'Overall Qual', 'Overall Cond', 'Year Built', 'Year Remod/Add', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Full Bath', 'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd', 'Fireplaces', 'Wood Deck SF', 'Open Porch S

#### Float columns

We will first look at columns of float data type

In [9]:
float_columns = ['Lot Frontage', 'Mas Vnr Area', 'BsmtFin SF 1', 'BsmtFin SF 2', 
                 'Bsmt Unf SF', 'Total Bsmt SF', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Garage Yr Blt', 
                 'Garage Cars', 'Garage Area']

##### Basement

In [10]:
columns_to_look = ['BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', 'Bsmt Full Bath', 'Bsmt Half Bath']

From kaggle dictionary, Bsmt Full Bath and Bsmt Half Bath represents the numbers of full and half bath in the basement, a better data type should be integer. We will convert them to integer. Again, this function was created in notebook 1. We shall use them for this purpose

In [11]:
def change_dtype(dataframe, columns, to_type):
    for i in columns:
        dataframe[i] = dataframe[i].astype(to_type)

In [12]:
columns = ['Bsmt Full Bath', 'Bsmt Half Bath']
to_type = 'int64'

change_dtype(train, columns, to_type)
change_dtype(test, columns, to_type)

Total Bsmt SF seems to be calculated from BsmtFin SF 1, BsmtFin SF 2, Bsmt Unf SF. Let us make a check to confirm that this is the case. 

In [13]:
train[train['Total Bsmt SF'] != (train['BsmtFin SF 1'] + train['BsmtFin SF 2'] + train['Bsmt Unf SF'])]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice


We can create another column with values for the total finished basement square feet which shows the total area of basement which are finished. This value can be calculated from the sum of finished area of basement 1 and finished area of basement 2

In [14]:
train['Bsmt Total Fin SF'] = train['BsmtFin SF 1'] + train['BsmtFin SF 2']
test['Bsmt Total Fin SF'] = test['BsmtFin SF 1'] + test['BsmtFin SF 2']

In [15]:
# update float columns to track our progress
float_columns = [x for x in float_columns if x not in columns_to_look] 

##### Garage

In [16]:
columns_to_look = ['Garage Yr Blt', 'Garage Cars', 'Garage Area']

Garage Yr Blt shows the year the garage was built reflecting the same kind of data as Yr Sold which is integer. We will change the data type of Garage Yr Blt to integer

In [17]:
columns = ['Garage Yr Blt']
to_type = 'int64'

change_dtype(train, columns, to_type)
change_dtype(test, columns, to_type)

From Kaggle Data Dictionary, Garage Cars is the number of cars the garage can fit. This can be inferred from Garage Area which shows the size of the garage. We will drop Garage Cars 

In [18]:
train.drop(['Garage Cars'], axis = 1, inplace = True)
test.drop(['Garage Cars'], axis = 1, inplace = True)

float_columns = [x for x in float_columns if x not in columns_to_look] # update float columns to track our progress

#### Lot Area and Masonry Veneer Area

In [19]:
columns_to_look = ['Lot Frontage', 'Mas Vnr Area']

From Kaggle Dictionary, Lot Frontage is the Linear Feet of street connected to property and Mas Vnr Area is the Mansonry Area in sqaure feet. During data cleaning, we see that Mas Vnr Area's mode is 0.0, we shall keep both columns for now. 

In [20]:
float_columns = [x for x in float_columns if x not in columns_to_look] # update float columns to track our progress

We had completed looking at all columns with float data type. We can confirm this by calling the updated list

In [21]:
float_columns

[]

### Object Columns

Now we will look at columns with object data type. We will first create a list of the columns and update the list after we have completed looking into the column

In [22]:
object_columns = ['MS Zoning', 'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config', 'Land Slope', 
                  'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type', 'House Style', 'Roof Style', 'Roof Matl', 
                  'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual', 
                  'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2', 'Heating', 'Heating QC', 'Central Air', 
                  'Electrical', 'Kitchen Qual', 'Functional', 'Fireplace Qu', 'Garage Type', 'Garage Finish', 'Garage Qual', 
                  'Garage Cond', 'Paved Drive', 'Pool QC', 'Fence', 'Misc Feature', 'Sale Type']

#### Ordinal Encoding

From Kaggle dictionary we can determine that there are quite a number of quality columns with the same categories. We shall look at them and fill them with numeric value, there are also some columns which do not display quality, however, from the data dictionary and some general knowledge, we can determine that one category will be better than another. Example, more utilities is considered better than no utilities.

We will first create a function which can assist us in performing this as this will be a repeated operation

In [23]:
# function to replace quality columns with values from dictionary

def quality_columns_to_numeric(dataframe, columns, dictionary):
    #possible_values = dict_values.keys()
    for i in columns:
        dataframe[i].replace(dictionary, inplace = True)
        dataframe[i] = dataframe[i].astype('int64')

In [24]:
columns_to_look = ['Exter Qual', 'Exter Cond', 'Bsmt Qual', 'Bsmt Cond', 'Heating QC', 'Kitchen Qual', 'Fireplace Qu', 
                   'Garage Qual', 'Garage Cond', 'Pool QC']

In [25]:
dictionary = {'Ex': 5,
               'Gd' : 4,
               'TA' : 3,
               'Fa' : 2,
               'Po' : 1,
               'None' : 0 }

quality_columns_to_numeric(train, columns_to_look, dictionary)
quality_columns_to_numeric(test, columns_to_look, dictionary)

In [26]:
object_columns = [x for x in object_columns if x not in columns_to_look] # update object columns to track our progress

In [27]:
columns_to_look = ['Bsmt Exposure']
dictionary = {'Gd' : 4,
             'Av' : 3,
             'Mn' : 2,
             'No' : 1,
             'None' : 0}

quality_columns_to_numeric(train, columns_to_look, dictionary)
quality_columns_to_numeric(test, columns_to_look, dictionary)
object_columns = [x for x in object_columns if x not in columns_to_look] # update object columns to track our progress

It can be inferred that properties with more utilities are better sought after, fetching a better price. As such we will  consider this column as a quality column and fill them with the logical ranking

In [28]:
columns_to_look = ['Utilities']
dictionary = {'AllPub' : 4,
             'NoSewr' : 3,
             'NoSeWa' : 2,
             'ELO' : 1}

quality_columns_to_numeric(train, columns_to_look, dictionary)
quality_columns_to_numeric(test, columns_to_look, dictionary)
object_columns = [x for x in object_columns if x not in columns_to_look] # update object columns to track our progress

Similar to Utilities, It can be inferred that properties with better foundation are perceived to have better quality. As such we will  consider this column as a quality column and fill them with the logical ranking

In [29]:
columns_to_look = ['Foundation']
dictionary = {'BrkTil' : 6,
             'CBlock' : 5,
             'PConc' : 4,
             'Slab' : 3,
             'Stone' : 2,
             'Wood' : 1}

quality_columns_to_numeric(train, columns_to_look, dictionary)
quality_columns_to_numeric(test, columns_to_look, dictionary)
object_columns = [x for x in object_columns if x not in columns_to_look] # update object columns to track our progress

We can infer that paved drive way is better than partial driveway. And they are considered better than no driveway. As such we will see them as quality columns

In [30]:
columns_to_look = ['Paved Drive']
dictionary = {'Y' : 2,
             'P' : 1,
             'N' : 0,}

quality_columns_to_numeric(train, columns_to_look, dictionary)
quality_columns_to_numeric(test, columns_to_look, dictionary)
object_columns = [x for x in object_columns if x not in columns_to_look] # update object columns to track our progress

From Kaggle's data dictionary, Fence is a quality column although at one look at the dataset it do not seem obvious

In [31]:
columns_to_look = ['Fence']
dictionary = {'GdPrv' : 4,
             'MnPrv' : 3,
             'GdWo' : 2,
             'MnWw' : 1,
             'None' : 0}

quality_columns_to_numeric(train, columns_to_look, dictionary)
quality_columns_to_numeric(test, columns_to_look, dictionary)
object_columns = [x for x in object_columns if x not in columns_to_look] # update object columns to track our progress

Properties with Central Air are deemed to better in quality than properties without centra air. As such we will treat this column as a quality column 

In [32]:
columns_to_look = ['Central Air']
dictionary = {'Y' : 2,
             'N' : 1,}

quality_columns_to_numeric(train, columns_to_look, dictionary)
quality_columns_to_numeric(test, columns_to_look, dictionary)
object_columns = [x for x in object_columns if x not in columns_to_look] # update object columns to track our progress

Electrical shows the type of electrical system of the property, which is also qualifiable. We shall treat this column as a quality column

In [33]:
columns_to_look = ['Electrical']
dictionary = {'SBrkr' : 5,
             'FuseA' : 4,
             'FuseF' : 3,
             'FuseP' : 2,
             'Mix' : 1,}

quality_columns_to_numeric(train, columns_to_look, dictionary)
quality_columns_to_numeric(test, columns_to_look, dictionary)
object_columns = [x for x in object_columns if x not in columns_to_look] # update object columns to track our progress

Functional shows the functionality of the house. Its categories includes, Typical, Minor Damage 1, Minor Damage 2 all the way to salvage. We will fill these categories accordingly based on their grade

In [34]:
columns_to_look = ['Functional']
dictionary = {'Typ' : 8,
             'Min1' : 7,
             'Min2' : 6,
             'Mod' : 5,
             'Maj1' : 4,
             'Maj2' : 2,
             'Sev' : 2,
             'Sal' : 1}

quality_columns_to_numeric(train, columns_to_look, dictionary)
quality_columns_to_numeric(test, columns_to_look, dictionary)
object_columns = [x for x in object_columns if x not in columns_to_look] # update object columns to track our progress

Sale Type is also a quality column, the type of sale affects the price, for example, New will fetch a better price. As such we will qualify the categories

In [35]:
columns_to_look = ['Sale Type']
dictionary = {'New' : 10,
              'Con' : 9,
             'ConLI' : 8,
             'CWD' : 7,
              'VWD' : 6,
             'WD ' : 5,
             'COD' : 4,
             'ConLD' : 3,
             'Oth' : 2,
             'ConLw' : 1}

quality_columns_to_numeric(train, columns_to_look, dictionary)
quality_columns_to_numeric(test, columns_to_look, dictionary)
object_columns = [x for x in object_columns if x not in columns_to_look] # update object columns to track our progress

Garage Finish shows if the garage is finish or their level of finish. We will also treat this column as a quality column

In [36]:
columns_to_look = ['Garage Finish']
dictionary = {'Fin' : 3,
             'RFn' : 2,
             'Unf' : 1,
             'None' : 0}

quality_columns_to_numeric(train, columns_to_look, dictionary)
quality_columns_to_numeric(test, columns_to_look, dictionary)
object_columns = [x for x in object_columns if x not in columns_to_look] # update object columns to track our progress

#### One Hot Encoding

In [37]:
print(object_columns)

['MS Zoning', 'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type', 'House Style', 'Roof Style', 'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type', 'BsmtFin Type 1', 'BsmtFin Type 2', 'Heating', 'Garage Type', 'Misc Feature']


The remaining object columns do not seems to refer to data which reflects quality of the property. As such we will one hot encode them. After encoding them, we will print the shape of both data frames to check for any discrepencies

In [38]:
train = pd.get_dummies(train, columns = object_columns, drop_first = True)
train.shape

(2048, 201)

In [39]:
test = pd.get_dummies(test, columns = object_columns, drop_first = True)
test.shape

(879, 190)

There are mismatched columns after one hot encoding. This is due to some categories in a column may exist in one dataframe but not the other. For example for Train dataset Garage Finish Column, all 4 categories are present, as such all 4 categories are one hot encoded. However the same column in Test may only have 3 of the categories, and only these 3 categories are one hot encoded

We will check for columns present in one data frame and not in the other, then add the column in with value 0

In [40]:
train_cols = train.columns
test_cols = test.columns

for col in train_cols:
    if col not in test_cols:
        test[col] = 0
        test[col] = test[col].astype('uint8')

for col in test_cols:
    if col not in train_cols:
            train[col] = 0
            train[col] = train[col].astype('uint8')
            
print(train.shape)
print(test.shape)

(2048, 208)
(879, 208)


In [41]:
# dropping sale price from test as it should not be there
test.drop('SalePrice', axis = 1, inplace = True) 

### Integer Columns

Now we will look at columns with Integer data type. We will first create a list of the columns and update the list after we have completed looking into the column

In [42]:
integer_columns = ['Id', 'PID', 'MS SubClass', 'Lot Area', 'Overall Qual', 'Overall Cond', 'Year Built', 'Year Remod/Add', 
                   '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 
                   'Full Bath', 'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd', 'Fireplaces', 'Garage Yr Blt', 
                   'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val', 
                   'Mo Sold', 'Yr Sold', 'SalePrice']

#### Id, PID, Overall Qual, Sale Price

We will first look at the below mentioned columns

In [43]:
columns_to_look = ['Id', 'PID', 'Overall Qual', 'Overall Cond', 'SalePrice']

In [44]:
train['Overall Qual'].value_counts()

5     561
6     506
7     431
8     250
4     158
9      77
3      29
10     23
2       9
1       4
Name: Overall Qual, dtype: int64

In [45]:
train['Overall Cond'].value_counts()

5    1168
6     367
7     268
8     101
4      70
3      35
9      29
2       6
1       4
Name: Overall Cond, dtype: int64

Id, PID and Sale Price was checked at the start of this notebook, we can considered it looked. Overall Qual and Overall Cond are quality columns and of the right data type 

In [46]:
integer_columns = [x for x in integer_columns if x not in columns_to_look] 
# update integer columns to track our progress

#### Sizes

Now we will look at columns showing sizes. Size should be float as sizes should have decimal places if looked into at a greater degree. 

In [47]:
columns_to_look = ['Lot Area', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area','Wood Deck SF', 'Open Porch SF', 
                   'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area']

In [48]:
to_type = 'float64'

change_dtype(train, columns_to_look, to_type)
change_dtype(test, columns_to_look, to_type)
integer_columns = [x for x in integer_columns if x not in columns_to_look] # update object columns to track our progress

#### Baths

Lets look at baths. From Kaggle data dictionary, we are unable to infer much for this columns. They are of the correct data type and we shall keep them as it is for now.

In [49]:
columns_to_look = ['Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Electrical', 'Sale Type', 'Functional', 'Central Air']
integer_columns = [x for x in integer_columns if x not in columns_to_look] # update object columns to track our progress

In [50]:
print(integer_columns)

['MS SubClass', 'Year Built', 'Year Remod/Add', 'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd', 'Fireplaces', 'Garage Yr Blt', 'Misc Val', 'Mo Sold', 'Yr Sold']


#### Dates

Now lets look at columns related to dates.

In [51]:
columns_to_look = ['Year Built', 'Year Remod/Add', 'Garage Yr Blt', 'Mo Sold', 'Yr Sold']

From Kaggle Data Dictionary, Year Remod/Add will be the same as construction date if there is no remodelling or additions. We do not know if there will be any correlation towards sale price for now, as such we will keep both columns. Age of property when sold is something which may have more correlation, as such we will create 2 columns, got age of property when sold, and age of property after reconstruction when sold. 

The same can be said for garage year built, as such we will create a column for garage age when sold

Month sold is redundant, we will drop the column.

In [52]:
train['Age When Sold'] = train['Yr Sold'] - train['Year Built']
train['Age After Remodeled When Sold'] = train['Yr Sold'] - train['Year Remod/Add']
train['Age Garage When Sold'] = train['Yr Sold'] - train['Garage Yr Blt']

test['Age When Sold'] = test['Yr Sold'] - test['Year Built']
test['Age After Remodeled When Sold'] = test['Yr Sold'] - test['Year Remod/Add']
test['Age Garage When Sold'] = test['Yr Sold'] - test['Garage Yr Blt']

In [53]:
train.drop('Mo Sold', axis = 1, inplace = True)
test.drop('Mo Sold', axis = 1, inplace = True)
integer_columns = [x for x in integer_columns if x not in columns_to_look] # update object columns to track our progress

In [54]:
print(integer_columns)

['MS SubClass', 'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd', 'Fireplaces', 'Misc Val']


#### Rooms

Lets look at room related columns

In [55]:
columns_to_look = ['Bedroom AbvGr', 'TotRms AbvGrd']

Bedroom Abv Gr and TotRms Abv Grd which means number of bedroom above grade and total number of rooms above grade from Kaggle's data dictionary.

Total rooms above grade will include the number of bedroom above grade, as such we will drop Bedroom Abv Gr column

In [56]:
train.drop('Bedroom AbvGr', axis = 1, inplace = True)
test.drop('Bedroom AbvGr', axis = 1, inplace = True)
integer_columns = [x for x in integer_columns if x not in columns_to_look] # update object columns to track our progress

In [57]:
print(test.shape)
print(train.shape)

(879, 208)
(2048, 209)


#### Kitchen, Fireplaces, Misc Val

Lets look at Kitchen AbvGr, Fireplaces, Misc Val

In [58]:
columns_to_look = ['Kitchen AbvGr', 'Fireplaces', 'Misc Val']

Kitchen above grade are the number of kitchen above grade, Fireplaces are the number of fireplaces in the property and Misc Val are the value of miscelleneous features.

We shall leave Kitchen AbvGr and Fireplaces as it is, and convert Misc Val to a float as it equates to a monetary value

In [59]:
column = ['Misc Val']
to_type = 'float64'

change_dtype(train, column, to_type)
change_dtype(test, column, to_type)
integer_columns = [x for x in integer_columns if x not in columns_to_look] # update object columns to track our progress

#### MS SubClass

Last of the integer column will be to look at MS SubClass

In [60]:
columns_to_look = ['MS SubClass']

From Kaggle's data dictionary, MA SubClass identifies the type of dwelling involved in the sale. This in my opinion is a variable which will have impact on the sale price. 

We shall convert them into a string wth meaningful value, then perform a one hot encoding. After performing one hot encoding, we will compare the columns to ensure that all columns are the same

In [61]:
dictionary = {20 : '1-STORY 1946 & NEWER ALL STYLES',
              30 : '1-STORY 1945 & OLDER',
             40 : '1-STORY W/FINISHED ATTIC ALL AGES',
             45 : '1-1/2 STORY - UNFINISHED ALL AGES',
              50 : '1-1/2 STORY FINISHED ALL AGES',
             60 : '2-STORY 1946 & NEWER',
             70 : '2-STORY 1945 & OLDER',
             75 : '2-1/2 STORY ALL AGES',
             80 : 'SPLIT OR MULTI-LEVEL',
             85 : 'SPLIT FOYER',
             90 : 'DUPLEX - ALL STYLES AND AGES',
             120 : '1-STORY PUD (Planned Unit Development) - 1946 & NEWER',
             150 : '1-1/2 STORY PUD - ALL AGES',
             160 : '2-STORY PUD - 1946 & NEWER',
             180 : 'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER',
             190 : '2 FAMILY CONVERSION - ALL STYLES AND AGES'}

train['MS SubClass'].replace(dictionary, inplace = True)
test['MS SubClass'].replace(dictionary, inplace = True)

In [62]:
train = pd.get_dummies(train, columns = ['MS SubClass'], drop_first = True)
test = pd.get_dummies(test, columns = ['MS SubClass'], drop_first = True)

train_cols = train.columns
test_cols = test.columns

for col in train_cols:
    if col not in test_cols:
        test[col] = 0
        test[col] = test[col].astype('uint8')

for col in test_cols:
    if col not in train_cols:
            train[col] = 0
            train[col] = train[col].astype('uint8')
            
print(train.shape)
print(test.shape)

(2048, 223)
(879, 223)


In [63]:
test.drop('SalePrice', axis = 1, inplace = True) # dropping sale price from test as it should not be there
integer_columns = [x for x in integer_columns if x not in columns_to_look] # update object columns to track our progress

We should have completed looking at all integer columns. Lets confirm 

In [64]:
integer_columns

[]

## Final Check and Saving Data Frame

After cleaning all the data, lets make a final check to confirm that we have the same number of columns and data type for each dataframes and save them into another file 

In [65]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2048 entries, 0 to 2047
Columns: 223 entries, Id to MS SubClass_SPLIT OR MULTI-LEVEL
dtypes: float64(20), int64(39), uint8(164)
memory usage: 1.2 MB


In [66]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Columns: 222 entries, Id to MS SubClass_1-1/2 STORY PUD - ALL AGES
dtypes: float64(20), int64(38), uint8(164)
memory usage: 539.2 KB


Everything seems good, train has one more int64 column which is price, something which should not be in test. The number of data types is the same for both data frames. We shall save the dataframes as another csv after the first iteration of feature engineering and continue with more eda, feature engineering and feature selection in the next phase

In [67]:
train.to_csv('../datasets/train_featured.csv', index=False)
test.to_csv('../datasets/test_featured.csv', index=False)